In [ ]:
import netket as nk
import numpy as np
import math
import json

In [ ]:
#graph
size = (4, 4)
row_num,col_num = size
n_lattice = row_num * col_num
num = [ i for i in range(row_num * col_num)]

begin = [ i * col_num for i in range(row_num)]
end = [(i + 1) * col_num - 1 for i in range(row_num)]

edgesx = []
edgesy = []
edgesz = []

for i in num:
    if i in begin:
        edgesx.append([i, i + col_num - 1])
    else:
        edgesx.append([i, i - 1])

for i in num:
    if i in num [ :col_num]:
        edgesy.append([i, num[- (col_num - i)]])
    else:
        edgesy.append([i, i - col_num])

for i in num:
    if i in num [-col_num:-1]:
        edgesz.append([i, num[- (col_num*3 - i)]+1])
    elif i in end:
        if i==n_lattice-1:
            edgesz.append([0, i])
        else:
            edgesz.append([i, i + 1])     
    else:
        edgesz.append([i, i + col_num + 1])
        
edges = edgesx + edgesz + edgesy

graph = nk.graph.Graph(edges)
hi = nk.hilbert.Spin(s = 0.5, graph = graph)#Hilbert space obtained as tensor product of local spin states.
graph.nodes()#the number of nodes 

In [ ]:
edgesz# For example, a list of edges in the Z direction

# 哈密顿的定义

In [ ]:
#pauli matrix
sx = [[0, 1], [1, 0]]
sy = [[0, -1j], [1j, 0]]
sz = [[1, 0], [0, -1]]
#parameter
h=1
J=1
#g = J / h
ha = nk.operator.LocalOperator(hi)
for i in edges:
    ha += -J*nk.operator.LocalOperator(hi, np.kron(sz, sz), i)
sxx = -h* nk.operator.LocalOperator(hi, [sx] * n_lattice, [[i] for i in range(n_lattice)])
ha += sxx #This term is magnetization, which can be regarded as an operator Mx(g)

In [ ]:
ma = nk.models.RBM(alpha=1)

# We shall use an exchange Sampler which preserves the global magnetization (as this is a conserved quantity in the model)
sa = nk.sampler.MetropolisExchange(hilbert=hi, graph=graph, d_max = 2)

# Construct the variational state
vs = nk.vqs.MCState(sa, ma, n_samples=1008)

# We choose a basic, albeit important, Optimizer: the Stochastic Gradient Descent
opt = nk.optimizer.Sgd(learning_rate=0.01)

# Stochastic Reconfiguration
sr = nk.optimizer.SR(diag_shift=0.01)

# We can then specify a Variational Monte Carlo object, using the Hamiltonian, sampler and optimizers chosen.
# Note that we also specify the method to learn the parameters of the wave-function: here we choose the efficient
# Stochastic reconfiguration (Sr), here in an iterative setup
gs = nk.VMC(hamiltonian=ha, optimizer=opt, variational_state=vs, preconditioner=sr)


# Measuring Observables

以下物理量与哈密顿量参数的关系

In [ ]:
obs={}

# spin-structure factor:

$$
S^{xx}(\vec{k})=\frac{1}{N(N-1)}\sum_{l\neq j}e^{-i\vec{k}.(\vec{r_l}-\vec{r_j})}\langle\sigma_l^x\sigma_j^x\rangle,
$$
where $\vec{k}=(2\pi/3,0)$.$\to$ 计算量允许的话可以画一个自旋结构因子二维图：横坐标为$k_x\in [0, 2\pi]$, 纵坐标为$k_y\in [0, 2\pi]$

当然，也可以算$S^{yy}(\vec{k})$ or $S^{zz}(\vec{k})$

In [ ]:
def d1tod2(po, row_num, col_num):
    # po -> (px, py)
    if po < 0 or po >= row_num*col_num:
        print("Wrong Index!")
    else:
        return 0.5*((po-(po//col_num)*col_num+(po//col_num)/2)*2), (math.sqrt(3)/2)*(po//col_num)
    
def d1tod2pbc(po, row_num, col_num):
    # po -> (px, py)
    if po < 0 or po >= row_num*col_num:
        print("Wrong Index!")
    else:
        return 0.5*((po-(po//col_num)*col_num+(po//col_num)/2)*2)+col_num, (math.sqrt(3)/2)*(po//col_num)

In [ ]:
import cmath
####### ################ Structure FactorX ###########################################
msxsx = (np.kron(sx, sx)) 
sfx = []
sitesfx = []
for i in range(0, n_lattice):
    xi, yi = d1tod2(i, row_num, col_num)
    for k in range(i, n_lattice):
        if i==k:
            pass
        else:
            xk1, yk1 = d1tod2(k, row_num, col_num)
            xk2, yk2 = d1tod2pbc(k, row_num, col_num)
#             dis = min(math.sqrt(abs(xi-xk1)**2+abs(yi-yk1)**2), math.sqrt(abs(xi-xk2)**2+abs(yi-yk2)**2))
            absx = min(abs(xi-xk1),abs(xi-xk2))
            sfx.append((cmath.exp(-complex(0,2*absx*3.14/3))*msxsx).tolist())
            sitesfx.append([i,k])

structure_factorx = nk.operator.LocalOperator(hi, sfx, sitesfx)
obs.update(StructureFactorX=structure_factorx)

# Magnization:

$$ M=\sqrt{\sum_\gamma M_\gamma^2}$$
where,$$M_\gamma=\frac{1}{N}\sum_{i}^N\langle\sigma_i^\gamma\rangle,\quad \gamma=x, y, z$$

In [ ]:
Mx = -h* nk.operator.LocalOperator(hi, [sx] * n_lattice, [[i] for i in range(n_lattice)])
My = -h* nk.operator.LocalOperator(hi, [sy] * n_lattice, [[i] for i in range(n_lattice)])
Mz = -h* nk.operator.LocalOperator(hi, [sz] * n_lattice, [[i] for i in range(n_lattice)])
obs.update(MX=Mx,MY=My,MZ=Mz)

# Coherence

$$C_{l_1}=\sum_{i\neq j}|\rho_{ij}|$$

In [ ]:
#############definition of the two-partite density element##########################
ketvec = []
for i in range(4):
    kettrans = [[0],[0],[0],[0]]
    kettrans[i] = [1]
    ketvec.append(kettrans)
   
    
bravec = []
for i in range(4):
    bratrans = [0,0,0,0]
    bratrans[i] = 1
    bravec.append(bratrans)

# only act on x-direction edges! -> Cl1_x. You can also act on all edges: for k in range(len(edges)):...
rhoterms = []
for i in range(4):
    for j in range(4):
        for k in range(len(edgesx)):
            rhoterms.append((np.kron(ketvec[i],bravec[j])/len(edgesx)).tolist())    

rhoindexx = []
for i in range(4):
    for j in range(4):
        rhoindexx.append('rhox%d%d'%(i,j)) # 2*2 Matrix's index

In [ ]:
############################ coherence ############################################   
for i in range(16):
    rhoterms_factorx = nk.operator.LocalOperator(hi,rhoterms[i*len(edgesx):(i+1)*len(edgesx)],edgesx)
#     gs.add_observable(rhoterms_factorx, rhoindexx[i])
    obs.update({rhoindexx[i]:rhoterms_factorx})

In [ ]:
obs

In [139]:
gs.run(out='test1.00', n_iter=600, obs=obs)

100%|██████████| 600/600 [01:16<00:00,  7.85it/s, Energy=-inf ± nan [σ²=nan]]                                                                                                                                                      


(JsonLog('test0.90', mode=write, autoflush_cost=0.005)
   Runtime cost:
   	Log:    0.31812548637390137
   	Params: 0.003862142562866211,)

# Plot

In [140]:
#coherence
json.load(open)
cox = []
for h in np.arange(0, 1.01, 0.1):
    data = json.load(open("<text%0.2f>.log"%(h)))
    iters = []
    energy = []
#     rho00x=[]
#     rho11x=[]
#     rho22x=[]
#     rho33x=[]
    rho01x = []
    rho02x = []
    rho03x = []
    rho10x = []
    rho12x = []
    rho13x = []
    rho20x = []
    rho21x = []
    rho23x = []
    rho30x = []
    rho31x = []
    rho32x = []
    print("J=%0.2f is loadded"%(J))
    for iteration in data["Output"]:
        iters.append(iteration["Iteration"])
        energy.append(iteration["Energy"]["Mean"])
#         rho00x.append(iteration["rhox00"]["Mean"])
#         rho11x.append(iteration["rhox11"]["Mean"])
#         rho22x.append(iteration["rhox22"]["Mean"])
#         rho33x.append(iteration["rhox33"]["Mean"])        
        rho01x.append(iteration["rhox01"]["Mean"])
        rho02x.append(iteration["rhox02"]["Mean"])
        rho03x.append(iteration["rhox03"]["Mean"])
        rho10x.append(iteration["rhox10"]["Mean"])
        rho12x.append(iteration["rhox12"]["Mean"])
        rho13x.append(iteration["rhox13"]["Mean"])
        rho20x.append(iteration["rhox20"]["Mean"])
        rho21x.append(iteration["rhox21"]["Mean"])
        rho23x.append(iteration["rhox23"]["Mean"])
        rho30x.append(iteration["rhox30"]["Mean"])
        rho31x.append(iteration["rhox31"]["Mean"])
        rho32x.append(iteration["rhox32"]["Mean"])
    ax=np.abs(np.mean(rho01x[-20:]))+np.abs(np.mean(rho02x[-20:]))+np.abs(np.mean(rho03x[-20:]))+np.abs(np.mean(rho10x[-20:]))+np.abs(np.mean(rho12x[-20:]))+np.abs(np.mean(rho13x[-20:]))+np.abs(np.mean(rho20x[-20:]))+np.abs(np.mean(rho21x[-20:]))+np.abs(np.mean(rho23x[-20:]))+np.abs(np.mean(rho30x[-20:]))+np.abs(np.mean(rho31x[-20:]))+np.abs(np.mean(rho32x[-20:]))
    cox.append(ax)

x=np.arange(0, 1.01, 0.1)
fig, ax = plt.subplots()
ax.plot(x, cox)
ax.set_xlabel('h')
ax.set_ylabel("Coherence")
plt.show()

AttributeError: 'builtin_function_or_method' object has no attribute 'read'